In [16]:
import time
import tracemalloc
from collections import deque
import heapq
import sys

In [17]:
# --- CLASE PRINCIPAL DEL JUEGO ---
class LightsOutGame:
    def __init__(self, board_file):
        self.rows = 0
        self.cols = 0
        self.initial_board = self.load_board(board_file)

    def load_board(self, filename):
        with open(filename, 'r') as f:
            lines = [line.strip() for line in f.readlines()]

        self.rows = len(lines)
        self.cols = len(lines[0])
        board = []
        for r in range(self.rows):
            row = []
            for c in range(self.cols):
                # O = Encendido (1), - = Apagado (0), X = Sin bombilla (-1)
                char = lines[r][c]
                if char == 'O': val = 1
                elif char == '-': val = 0
                else: val = -1 # X
                row.append(val)
            board.append(tuple(row))
        return tuple(board)

    def is_goal(self, board):
        # El objetivo es que todas las luces (que no sean X) estén apagadas (0)
        for r in range(self.rows):
            for c in range(self.cols):
                if board[r][c] == 1:
                    return False
        return True

    def get_successors(self, board):
        successors = []
        # Orden de visita especificado: UDLR (Arriba, Abajo, Izquierda, Derecha)
        # Pero aquí elegimos qué CELDA tocar. Iteramos en orden de lectura (0,0) a (n,m)
        for r in range(self.rows):
            for c in range(self.cols):
                if board[r][c] != -1: # Si no es una celda vacía (X)
                    new_board = self.toggle(board, r, c)
                    successors.append(((r, c), new_board))
        return successors

    def toggle(self, board, r, c):
        # Convertir a lista de listas para mutar
        new_b = [list(row) for row in board]

        # Movimientos relativos: Centro, Arriba, Abajo, Izq, Der
        moves = [(0, 0), (-1, 0), (1, 0), (0, -1), (0, 1)]

        for dr, dc in moves:
            nr, nc = r + dr, c + dc
            if 0 <= nr < self.rows and 0 <= nc < self.cols:
                if new_b[nr][nc] != -1: # No afectar celdas X
                    new_b[nr][nc] = 1 - new_b[nr][nc] # Toggle 0 <-> 1

        return tuple(tuple(row) for row in new_b)

In [18]:
def heuristic_1(state):
    # Conteo de luces encendidas
    return bin(state).count('1')

def heuristic_2(state, rows, cols):
    # Ponderación por posiciones críticas (bordes y esquinas)
    cost = 0
    for idx in range(rows * cols):
        if (state & (1 << idx)):
            r, c = divmod(idx, cols)
            weight = 1
            if (r == 0 or r == rows-1) or (c == 0 or c == cols-1):
                weight = 1.5
            cost += weight
    return cost

def heuristic_3(state, rows, cols):
    # Penalización por luces aisladas
    cost = 0
    for idx in range(rows * cols):
        if (state & (1 << idx)):
            r, c = divmod(idx, cols)
            cost += 1
            isolated = True
            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nr, nc = r + dr, c + dc
                if 0 <= nr < rows and 0 <= nc < cols:
                    if (state & (1 << (nr * cols + nc))):
                        isolated = False
                        break
            if isolated: cost += 2
    return cost


In [19]:
def solve(game, algorithm='BFS', heuristic_weights=None, ram_limit=45240, max_depth=None):
    start_time = time.time()
    tracemalloc.start()

    start_node = game.initial_board
    nodes_expanded = 0
    max_search_depth = 0
    tie_breaker = 0

    if algorithm == 'BFS':
        frontier = deque([(start_node, [])])
    elif algorithm == 'DFS':
        frontier = [(start_node, [])]
    elif algorithm == 'AST':
        frontier = [(0, 0, start_node, [])]

    explored = {start_node}

    while frontier:
        _, peak = tracemalloc.get_traced_memory()
        if (peak / 1048576) > ram_limit:
            tracemalloc.stop()
            return {"error": "Memory limit reached", "running_time": time.time()-start_time, "max_ram_usage": peak/1048576}

        if algorithm == 'AST':
            f, _, current_state, path = heapq.heappop(frontier)
        elif algorithm == 'BFS':
            current_state, path = frontier.popleft()
        else:  # DFS
            current_state, path = frontier.pop()

        if game.is_goal(current_state):
            end_time = time.time()
            _, final_peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            return {
                "path_to_goal": path,
                "cost_of_path": len(path),
                "nodes_expanded": nodes_expanded,
                "search_depth": len(path),
                "max_search_depth": max_search_depth,
                "running_time": end_time - start_time,
                "max_ram_usage": final_peak / 1048576
            }

        nodes_expanded += 1
        max_search_depth = max(max_search_depth, len(path))

        if algorithm == 'DFS' and max_depth is not None and len(path) >= max_depth:
            continue

        successors = game.get_successors(current_state)

        if algorithm == 'DFS':
            successors.reverse()

        for move, next_state in successors:
            if next_state not in explored:

                explored.add(next_state)

                if algorithm == 'AST':
                    w1, w2, w3 = heuristic_weights if heuristic_weights else (1, 1, 1)
                    g = len(path) + 1
                    h = (heuristic_1(next_state) * w1) + \
                        (heuristic_2(next_state, game.rows, game.cols) * w2) + \
                        (heuristic_3(next_state, game.rows, game.cols) * w3)
                    tie_breaker += 1
                    heapq.heappush(frontier, (g + h, tie_breaker, next_state, path + [move]))
                else:
                    frontier.append((next_state, path + [move]))

    tracemalloc.stop()
    return {"error": "No solution found"}


In [20]:
def generate_output(filename, algorithm, weights=None):
    game = LightsOutGame(filename)

    result = solve(game, algorithm, weights, ram_limit=45240, max_depth=30)

    if "error" in result:
        output_str = (
            f"Algorithm: {algorithm} (Weights: {weights})\n"
            f"Error: {result['error']}\n"

            f"running_time: {result.get('running_time', 0):.8f}\n"
            f"max_ram_usage: {result.get('max_ram_usage', 0):.8f}\n"

            "--------------------------------------------------\n"
        )
    else:
        output_str = (
            f"Algorithm: {algorithm} (Weights: {weights})\n"
            f"path_to_goal: {result['path_to_goal']}\n"
            f"cost_of_path: {result['cost_of_path']}\n"
            f"nodes_expanded: {result['nodes_expanded']}\n"
            f"search_depth: {result['search_depth']}\n"
            f"max_search_depth: {result['max_search_depth']}\n"
            f"running_time: {result['running_time']:.8f}\n"
            f"max_ram_usage: {result['max_ram_usage']:.8f}\n"
            "--------------------------------------------------\n"
        )
    print(output_str)
    with open("output.txt", "a") as f:
        f.write(output_str)

In [21]:
def board_to_str(board):

    out_lines = []
    for r, row in enumerate(board):
        parts = []
        for c, v in enumerate(row):
            if v == -1:
                parts.append("X")
            elif v == 1:
                parts.append("O")
            else:
                parts.append("-")
        out_lines.append(" ".join(parts))
    return "\n".join(out_lines)

def print_board(board):

    cols = len(board[0])
    header = "    " + " ".join(f"{c:>2}" for c in range(cols))
    print(header)
    print("    " + "---" * cols)
    for r, row in enumerate(board):
        pretty = []
        for v in row:
            if v == -1: pretty.append(" X")
            elif v == 1: pretty.append(" O")
            else: pretty.append(" -")
        print(f"{r:>2} |" + "".join(pretty))
    print()

def parse_move(s):
    """
    Acepta:
      - "r c"  (ej: 2 3)
      - "r,c"  (ej: 2,3)
    Devuelve (r,c) o None si no es válido.
    """
    s = s.strip()
    if not s:
        return None
    s = s.replace(",", " ")
    parts = [p for p in s.split() if p]
    if len(parts) != 2:
        return None
    try:
        r = int(parts[0]); c = int(parts[1])
        return (r, c)
    except ValueError:
        return None

def play_console(level_file):
    """
    Juego por consola:
      - Escribe "r c" para tocar una celda.
      - Comandos: help, reset, quit
    """
    game = LightsOutGame(level_file)
    board = game.initial_board
    moves = []

    print("\n=== LIGHTS OUT (Consola) ===")
    print(f"Nivel: {level_file}  Tamaño: {game.rows}x{game.cols}")
    print('Comandos: "r c", help, reset, quit\n')

    while True:
        print_board(board)

        if game.is_goal(board):
            print(f"¡Ganaste! Movimientos: {len(moves)}")
            if moves:
                print("Tus clicks:", moves)
            break

        cmd = input("> ").strip()
        if not cmd:
            continue

        low = cmd.lower()

        if low in ("q", "quit", "exit"):
            print("Saliendo del juego.")
            break

        if low in ("h", "help", "?"):
            print(
                "Uso:\n"
                "  r c     -> toca (r,c)\n"
                "  reset   -> vuelve al estado inicial del nivel\n"
                "  quit    -> salir\n"
            )
            continue

        if low == "reset":
            board = game.initial_board
            moves.clear()
            continue

        # Interpretar como movimiento: "r c" o "r,c"
        mv = parse_move(cmd)
        if mv is None:
            print('Entrada inválida. Escribe "help" para ver comandos.')
            continue

        r, c = mv
        if not (0 <= r < game.rows and 0 <= c < game.cols):
            print("Fuera de rango.")
            continue
        if board[r][c] == -1:
            print("Esa celda es X (sin bombilla). No se puede tocar.")
            continue

        board = game.toggle(board, r, c)
        moves.append((r, c))


In [22]:
# CONFIGURACIÓN: Cambia el nombre del archivo y el algoritmo aquí
if __name__ == "__main__":

    niveles_medianos = ["5x5.1facil.txt", "5x5.2facil.txt"]

    for nivel in niveles_medianos:
        print(f"=== PROCESANDO NIVEL: {nivel} ===")
        # 1. Ejecutar BFS
        generate_output(nivel, "BFS")

        # 2. Ejecutar DFS
        generate_output(nivel, "DFS")

        # 3. Ejecutar A* con pesos iguales
        generate_output(nivel, "AST", weights=[1, 1, 1])

        # 4. Ejecutar A* con configuración de pesos 1
        generate_output(nivel, "AST", weights=[2, 0.5, 0.5])

        # 5. Ejecutar A* con configuración de pesos 2
        generate_output(nivel, "AST", weights=[0.5, 2, 0.5])

        # 6. Ejecutar A* con configuración de pesos 3
        generate_output(nivel, "AST", weights=[0.5, 0.5, 2])
        print("\n")

=== PROCESANDO NIVEL: 5x5.1facil.txt ===
Algorithm: BFS (Weights: None)
path_to_goal: [(0, 0), (0, 1), (1, 0), (1, 2), (1, 4), (2, 1), (2, 4), (4, 1), (4, 2), (4, 4)]
cost_of_path: 10
nodes_expanded: 3569409
search_depth: 10
max_search_depth: 10
running_time: 1299.73644495
max_ram_usage: 3378.50803280
--------------------------------------------------

Algorithm: DFS (Weights: None)
path_to_goal: [(0, 0), (0, 1), (0, 2), (0, 0), (0, 3), (0, 1), (0, 0), (0, 4), (0, 1), (0, 2), (0, 0), (0, 1), (1, 0), (0, 0), (0, 3), (0, 1), (0, 0), (0, 2), (0, 1), (1, 3), (3, 4), (4, 1), (1, 0), (2, 1), (0, 1), (4, 0), (2, 4), (3, 2), (1, 3), (3, 0)]
cost_of_path: 30
nodes_expanded: 1014742
search_depth: 30
max_search_depth: 30
running_time: 61.29176688
max_ram_usage: 496.69018555
--------------------------------------------------



TypeError: 'tuple' object cannot be interpreted as an integer

In [ ]:
if __name__ == "__main__":
  level = "5x5.1facil.txt"
  play_console(level)
